In [2]:
# installation files
pip3 install torch==1.8.2+cu111 torchvision==0.9.2+cu111 torchaudio===0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
pip3 install transformers

SyntaxError: invalid syntax (Temp/ipykernel_15396/1427826365.py, line 2)

In [3]:
pip3 install transformers

SyntaxError: invalid syntax (Temp/ipykernel_15396/4046715967.py, line 1)

In [1]:
import sys
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from collections import defaultdict
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

ModuleNotFoundError: No module named 'transformers'

In [260]:
class SentimentClassifier(nn.Module):

  def __init__(self):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained('bert-base-cased')
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, 3)
  
  def forward(self, input_ids, attention_mask):
    bert_out = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    pooled_output = bert_out['pooler_output']
    # print(pooled_output)
    output = self.drop(pooled_output)
    prob = F.softmax(self.out(output))
    return prob

In [261]:
MAX_LEN = 250

In [262]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [263]:
# model = torch.load('../models/bert_sentiment_model.pt', map_location=torch.device('cpu'))
model = SentimentClassifier()
model.load_state_dict(torch.load('../models/bert_sentiment_model.pt', map_location=torch.device('cpu')))

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [264]:
def get_sentence_sentiment(s):
    text = s
    encoded_review = tokenizer.encode_plus(
    text,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=True,
    return_tensors='pt',
    )
    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)

    output = model(input_ids, attention_mask)
    return int(torch.argmax(output) - 1)

In [265]:
get_sentence_sentiment("I hate this")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\ProgramData\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2226: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
C:\Users\KENNET~1\AppData\Local\Temp/ipykernel_4408/1253271525.py:17: UserWarning: Implicit dime

-1

In [266]:
# variables for testing, replace with actual data openings after

df_ss = pd.DataFrame({
    'sid': [1, 2, 3],
    'sentence': ["I hate this", "I love this", "The sky is blue"]
})

df_sr = pd.DataFrame({
    'sid': [1, 2, 3],
    'rid': [1,1,2]
})

df_st = pd.DataFrame({
    'sid': [1, 2, 3],
    'x1': [1,1,2],
    'x2' : [2, 4, 8]
})

df_masks = pd.DataFrame({
    'params': ['quality', 'price'],
    'use_weighted': [1, 0],
    'x1': [.75,.25],
    'x2' : [.2, .9]
})
df_masks = df_masks.set_index('params')


assume that these will be files from the tf idf analysis

sid, sentence

rid, restaurant

rid, sid

sid, tfidf vector

 this will be the process
 
sid, sentence, sentiment

sid, sentence, sentiment, tfidf vector

create two, one for the sentiment * tfidf vector, the other keep as is.

sum per restaurant, have two copies

normalize per restaurant

mask it for the final parameter                                                                                                                         

In [267]:
# get the sentiments for all sentences, then save, since this will take a while
df_ss['sentiment'] = df_ss['sentence'].map(lambda x: get_sentence_sentiment(x))
df_ss

# save the sentiments per sentence (since this might take a while to run)
df_ss.to_csv("../script_data/sentence_sentiments.csv")

C:\Users\KENNET~1\AppData\Local\Temp/ipykernel_4408/1253271525.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = F.softmax(self.out(output))


In [268]:
# combine the sid, sentence, sentiment dataframe, with the tf idf vectors
df_sst = pd.merge(df_ss, df_st)
df_sst

,sid,sentence,sentiment,x1,x2
0,1,I hate this,-1,1,2
1,2,I love this,1,1,4
2,3,The sky is blue,0,2,8


In [269]:
# make a copy of the dataframe, and for the weighted one, mmultiply the tfidf vector by the sentiment
df_sst_w = df_sst.copy()
df_sst_w.loc[:, ~df_sst_w.columns.isin(['sentence', 'sentiment', 'sid'])] = df_sst_w.loc[:, ~df_sst_w.columns.isin(['sentence', 'sentiment', 'sid'])].multiply(df_sst_w['sentiment'], axis="index")
df_sst_w

,sid,sentence,sentiment,x1,x2
0,1,I hate this,-1,-1,-2
1,2,I love this,1,1,4
2,3,The sky is blue,0,0,0


In [270]:
# merge the two tables with the sid, rid table
df_rsst = pd.merge(df_sr, df_sst)
df_rsst_w = pd.merge(df_sr, df_sst_w)
df_rsst

,sid,rid,sentence,sentiment,x1,x2
0,1,1,I hate this,-1,1,2
1,2,1,I love this,1,1,4
2,3,2,The sky is blue,0,2,8


In [271]:
# by rid, sum up the tf idf vectors, get rid of the sid, since it longer makes sense summed
df_rt = df_rsst.groupby(['rid']).sum()
df_rt = df_rt.drop(['sid', 'sentiment'], axis = 1)

df_rt_w = df_rsst_w.groupby(['rid']).sum()
df_rt_w = df_rt_w.drop(['sid', 'sentiment'], axis = 1)
df_rt

,x1,x2
rid,,
1,2,6
2,2,8


In [272]:
# now we have the aggregated data per restaurant, now we want to normalize and apply the masks
df_rt = df_rt.div(df_rt.sum(axis=1), axis=0)
df_rt_w = df_rt_w.div(df_rt_w.sum(axis=1), axis=0)
df_rt

,x1,x2
rid,,
1,0.25,0.75
2,0.20,0.80


In [273]:
# fill all the nan (all row 0), with 0s
df_rt = df_rt.fillna(0)
df_rt_w = df_rt_w.fillna(0)
df_rt

,x1,x2
rid,,
1,0.25,0.75
2,0.20,0.80


In [274]:
# apply the masks using matrix multiplication
# first, separate mask by uses weighted or not
df_masks_w = df_masks[df_masks['use_weighted'] == 1].drop('use_weighted', axis = 1)
df_masks = df_masks[df_masks['use_weighted'] == 0].drop('use_weighted', axis = 1)

df_r_features = df_masks.dot(df_rt.T).T
df_r_features_w = df_masks_w.dot(df_rt_w.T).T

df_r_features = df_r_features.merge(df_r_features_w, on = 'rid')
df_r_features

params,price,quality
rid,,
1,0.7375,0.2
2,0.7700,0.0


In [275]:
# save the features per restaurant
df_r_features.to_csv("../script_data/rest_features.csv")